In [ ]:
import json
from IPython.display import clear_output, HTML, display
import ipywidgets.widgets as widgets
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

In [ ]:
with open('manipulator-track-01.json', 'r') as hf:
    track = []
    for line in hf:
        try:
            frame = json.loads(line)
            track.append(frame)
        except Exception as e:
            print(f'broken frame {e}')

In [ ]:
len(track)

In [ ]:
print(json.dumps(track[105], indent=4, sort_keys=True))

In [ ]:
%matplotlib notebook

In [ ]:
plt.style.use('ggplot')
plt.figure(figsize=(20, 10))
servo_name = ['m-0', 'm-1', 'm-2', 'm-claw']
line_names = ['time', 'intended', 'real', 'diff', 'servo', 'ros_ts_lag']
for index in range(1, 5):
    plt.subplot(2, 2, int(index))
    lines = {field: list() for field in line_names}
    for it in track:
        lines['time'].append(it['ts'])
        for field in ['intended', 'real']:
            lines[field].append(it['manipulator']['angles'][index-1][field])
        if lines['real'][-1] is None:
            lines['diff'].append(None)
        else:
            lines['diff'].append(lines['intended'][-1] - lines['real'][-1])
        lines['servo'].append(it['manipulator']['servo_positions'][str(index-1)])
        ros_ts = it['manipulator']['raw_angles'][servo_name[index-1]]['ts']
        if ros_ts:
            lines['ros_ts_lag'].append(it['ts'] - ros_ts)
        else:
            lines['ros_ts_lag'].append(None)
    time_base = lines['time'][0]
    lines['time'] = [x - time_base for x in lines['time']]
    for line_name in line_names:
        if line_name == 'time':
            continue
        plt.plot(lines['time'], lines[line_name], label=line_name)

    plt.grid(True)
    plt.legend()
    plt.title(str(int(index)))
    plt.tight_layout()
plt.suptitle('By servo')
plt.show()